# Pre-trained Language Models: SubTask C 
## [8 Marks]

In this assignment, you will work on the [ComVE](https://competitions.codalab.org/competitions/21080) shared task that was part of SemEval-2020. The task aims to evaluate whether a system can distinguish if a natural language statement makes sense to humans or not and provide a reason. **ConVE** includes three subtasks that require models to acquire and apply commonsense knowledge. In this notebook you will focus on **SubTask C**:

- Given a statement that does not make sense, generate the reason why this statement does not make sense. For each nonsensical statement, three valid reasons are given as reference:

     *Statement*: He put an elephant into the fridge.  
     *Reason A*: An elephant is much bigger than a fridge.  
     *Reason B*: A fridge is much smaller than an elephant.  
     *Reason C*: Most of the fridges aren't large enough to contain an elephant.

     This subtask can be approached as a Sequence-to-Sequence problem where the input is the nonsensical statement and the output is a valid reason.

You will fine-tune a Pre-trained Language Model with [Transformers](https://huggingface.co/docs/transformers/index) library that provides a set of tools for fine-tunning and deploying a wide variety of Pre-trained Language Models. The [Hugging Face Hub](https://huggingface.co/models) allows you to explore all the models supported by **Transformers** and even share your own models with the community. In this assignment, you will work with [BART](https://huggingface.co/docs/transformers/model_doc/bart), a pre-trained Sequence-to-Sequence model.

Fine-tuning a Pre-trained Language Model usually requires a great amount of time and computational resources. Your personal computer will not be enough. In order to complete the assignment, you can work with a reduced version of the dataset and the base version of **BART**:

In [1]:
shrink_dataset = True
base_model = True
colab = False

Although the value of these variables do not affect the tests that will evaluate your code, the output examples distributed throughout this notebook are based on a `shrink_dataset` and a `base_model` variables set as `True`, and a `colab` variable set as `False`.

If you want to perform a full training of the model to obtain its real performance, you can use a cloud service like [Google Colab](https://colab.research.google.com/). **Colab** is a **Jupyter** notebook environment that supports both GPU and TPU instances, allowing training large scale Deep Learning models. Set the `shrink_dataset` and a `base_model` variables to `False`, the `colab` variable to `True`, and follow the instructions provided to you to run the notebook in **Colab**.

In [2]:
if colab:
    ! pip install transformers datasets evaluate
    import os
    if not os.path.exists("SemEval2020-Task4-Data/ALL data/Training  Data/subtaskC_data_all.csv"):
        ! git clone https://github.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation.git SemEval2020-Task4-Data

You will use the following objects and functions:

In [3]:
import pandas as pd
import evaluate
from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM, 
                          Seq2SeqTrainingArguments, Seq2SeqTrainer, 
                          DataCollatorForSeq2Seq, enable_full_determinism)

When working with Neural Networks, there are a large number of random operations such as initializing the weights of the network, shuffling the data for training, or choosing samples. This causes that different training runs of the same model can lead to different results. To ensure reproducibility, i.e. obtaining the same results in the different runs, the random number generator must be initialized with a fixed value known as seed. In Transformers, this can be done as follows:

In [4]:
enable_full_determinism(seed=42)

> **Note!** With models as complex as Neural Networks, reproducibility is susceptible to factors such as software versions or the hardware on which the models are run. Even with seed initialization, there may be slight differences in the results.

Working with Neural Networks also involves defining a number of hyperparameters that set the configuration of the model. Finding the appropriate hyperparameter values requires training the model with different combinations and testing them on the development set. This hyperparameter tuning is a costly process that needs multiple rounds of experimentation. However, for this assignments, you will use the following values:

In [5]:
epochs = 3  # Number of epochs to train the model
train_batch_size = 8  # Number of examples used per gradient update
learning_rate = 1e-5  # The learning rate for the optimizer
max_length = 25  # Maximum lenght of the input sequence
output_dir = "modelC"  # The output directory where the model will be written

> **Note!** The notebook for this assignment provides very little guidance. You are expected to refer to the [documentation](https://huggingface.co/docs) for details on how to solve the exercises.

## Loading the Pre-trained Model - [1 Mark]

The first step you must perform in this assignment is to load the model and its corresponding tokenizer using the classes imported above.

In [6]:
def load_model(model_name):   #[1 Mark]
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

In [7]:
model_name = "facebook/bart-base" if base_model else "facebook/bart-large"
model, tokenizer = load_model(model_name)

## Data Pre-processing - [2 Marks]

The **ComVE** dataset consists of 10000 nonsensical statements for the train set, 997 statements for development and 1000 for test. Each nonsensical statements comes with with three reference valid reasons. You must load the three sets into three `DataFrames`. For the training and development splits, the `DataFrame` should contain three columns: the `id` of the nonsensical statement, a `FalseSent` column with the nonsensical statement and a `reason` column with the reference reasons. For the test set, the `DataFrame` should contain five columns: the `id` of the nonsensical statement, a `FalseSent` column with the nonsensical statement and three columns (`reason1`, `reason2` and `reason3`) containing each of the reference reasons.

Train DataFrame:

|       |   id | FalseSent                                         | reason                                                                         |
|------:|-----:|:--------------------------------------------------|:-------------------------------------------------------------------------------|
|   769 |  769 | Computers is an ingredient used in preparing food | Computers are not used for food and they are not edible                        |
| 10769 |  769 | Computers is an ingredient used in preparing food | Computer is not something that can be used in preparing food.                  |
| 20769 |  769 | Computers is an ingredient used in preparing food | You cannot eat a computer                                                      |
|   888 |  888 | he did hear music in his cooling glass            | cooling glass can not play the song, it's not a electronic thing to play music |
| 10888 |  888 | he did hear music in his cooling glass            | Glass does not produce music.                                                  |
| 20888 |  888 | he did hear music in his cooling glass            | Any sound that might be made by a cooling glass is not music.                  |

Test DataFrame:

|     |   id | FalseSent                                      | reason1                                                  | reason2                                                | reason3                                                            |
|----:|-----:|:-----------------------------------------------|:---------------------------------------------------------|:-------------------------------------------------------|:-------------------------------------------------------------------|
|  76 | 1280 | Beer that is drunk by humans is white          | Beer is made of barley and it is a yellow drink          | A beer that is drunk by humans is not white.           | Beer is brown                                                      |
| 101 |  860 | eating trash food every day makes you stronger | eating trash food every day makes your body fat and weak | eating trash food every day is bad for your health     | Trash food could be contaminated                                   |
| 136 |  777 | he put some cooking oil in his wine            | cooking oil will destroy the taste of the wine           | Cooking oil does not go in wine                        | Cooking oil does not taste nice and therefore would ruin the wine. |
| 174 |  570 | Lobsters live in the mountains                 | Lobsters needs water to live                             | Lobsters live in the sea.                              | Lobsters live in the sea, not the mountains                        |
| 210 | 1929 | the clock shows animals                        | the clock is used to show the time to people             | Clocks are required to tell the time, not show animals | a clock shows the time not animals                                 |
| 235 | 1619 | she put the giraffe in the freezer             | A giraffe is much bigger than the freezer                | There is no way a giraffe is fitting in the freezer.   | A giraffe is too big to be put in a freezer.                       |

In [8]:
def load_data(data_csv, answers_csv, is_test=False):   # 1 Mark
    
    data_csv = pd.read_csv(data_csv, names=["id", "FalseSent"], header=0)

    if is_test:
        answers_df = pd.read_csv(answers_csv, names=["id", "reason1", "reason2", "reason3"], header=None)
        merged_df = pd.merge(data_csv, answers_df, on="id")
    else:
        answers_csv = pd.read_csv(answers_csv, names=["id", "reason1", "reason2", "reason3"], header=0)
        result_df = pd.melt(answers_csv, id_vars=["id"], value_vars=["reason1", "reason2", "reason3"], value_name="reason")
        result_df = result_df.drop(["variable"], axis=1)
        merged_df = pd.merge(data_csv,result_df,on='id',how='inner')

    return merged_df

In [9]:
train_data_csv = "ALL data/Training  Data/subtaskC_data_all.csv"
train_answers_csv = "ALL data/Training  Data/subtaskC_answers_all.csv"
train_data = load_data(train_data_csv, train_answers_csv)
dev_data_csv = "ALL data/Dev Data/subtaskC_dev_data.csv"
dev_answers_csv = "ALL data/Dev Data/subtaskC_gold_answers.csv"
dev_data = load_data(dev_data_csv, dev_answers_csv)
test_data_csv = "ALL data/Test Data/subtaskC_test_data.csv"
test_answers_csv = "ALL data/Test Data/subtaskC_gold_answers.csv"
test_data = load_data(test_data_csv, test_answers_csv, True)
if shrink_dataset:
    idxs = train_data["id"].sample(frac=1, random_state=42).unique()[:30]
    train_data = train_data[train_data.id.isin(idxs)]
    idxs = dev_data["id"].sample(frac=1, random_state=42).unique()[:30]
    dev_data = dev_data[dev_data.id.isin(idxs)]
    idxs = test_data["id"].sample(frac=1, random_state=42).unique()[:30]
    test_data = test_data[test_data.id.isin(idxs)]
pd.set_option("display.max_colwidth", None)
print("Train DataFrame:")
display(train_data[:6])
print("Test DataFrame:")
display(test_data[:6])

Train DataFrame:


,id,FalseSent,reason
2307,770,The plane is flying between the planets.,The plane cannot go into space.
2308,770,The plane is flying between the planets.,Planes are require air to fly and there is no air in space.
2309,770,The plane is flying between the planets.,You can not fly a plane in space
2664,889,My car is a sphere,"Planets are spheres, but my car does not have a sphere as volume"
2665,889,My car is a sphere,Cars are not spherical
2666,889,My car is a sphere,Cars are not shaped like a sphere.


Test DataFrame:


,id,FalseSent,reason1,reason2,reason3
76,1280,Beer that is drunk by humans is white,Beer is made of barley and it is a yellow drink,A beer that is drunk by humans is not white.,Beer is brown
101,860,eating trash food every day makes you stronger,eating trash food every day makes your body fat and weak,eating trash food every day is bad for your health,Trash food could be contaminated
136,777,he put some cooking oil in his wine,cooking oil will destroy the taste of the wine,Cooking oil does not go in wine,Cooking oil does not taste nice and therefore would ruin the wine.
174,570,Lobsters live in the mountains,Lobsters needs water to live,Lobsters live in the sea.,"Lobsters live in the sea, not the mountains"
210,1929,the clock shows animals,the clock is used to show the time to people,"Clocks are required to tell the time, not show animals",a clock shows the time not animals
235,1619,she put the giraffe in the freezer,A giraffe is much bigger than the freezer,There is no way a giraffe is fitting in the freezer.,A giraffe is too big to be put in a freezer.


In [10]:
train_dataset = Dataset.from_pandas(train_data)
dev_dataset = Dataset.from_pandas(dev_data)
test_dataset = Dataset.from_pandas(test_data)
print("Train Dataset example:")
display(train_dataset[0])
print("Test Dataset example:")
display(test_dataset[0])

Train Dataset example:


{'id': 770,
 'FalseSent': 'The plane is flying between the planets.',
 'reason': 'The plane cannot go into space.',
 '__index_level_0__': 2307}

Test Dataset example:


{'id': 1280,
 'FalseSent': 'Beer that is drunk by humans is white',
 'reason1': 'Beer is made of barley and it is a yellow drink',
 'reason2': 'A beer that is drunk by humans is not white.',
 'reason3': 'Beer is brown',
 '__index_level_0__': 76}

The `Datasets` should be pre-processed following two different approaches. For the test `Dataset`, you must run the tokenizer on the `FalseSent` column and store the result in the `input_ids` and `attention_mask` fields. For the train and development `Datasets` you must also run the tokenizer on the `reason` column and store the resulting `input_ids` in the `labels` field. In all cases, the tokenizer must pad and truncate the sequences to the `max_length` value.

><pre>
>Train formated Dataset example:
>
>{'id': 769, 'FalseSent': 'Computers is an ingredient used in preparing food', 'reason': 'Computers are not used for food and they are not edible', '__index_level_0__': 769, 'input_ids': [0, 14721, 43990, 16, 41, 16181, 341, 11, 4568, 689, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [0, 14721, 43990, 32, 45, 341, 13, 689, 8, 51, 32, 45, 27532, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
>
>Test formated Dataset example:
>
>{'id': 1280, 'FalseSent': 'Beer that is drunk by humans is white', 'reason1': 'Beer is made of barley and it is a yellow drink', 'reason2': 'A beer that is drunk by humans is not white.', 'reason3': 'Beer is brown', '__index_level_0__': 76, 'input_ids': [0, 45562, 14, 16, 10789, 30, 5868, 16, 1104, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
></pre>

In [11]:
def preprocess_data(examples, tokenizer, max_length, is_test=False):   # [1 Mark]
    if is_test:
        inputs = tokenizer(examples["FalseSent"], truncation=True, max_length=max_length, padding="max_length")
        return inputs
    else:
        inputs = tokenizer(examples["FalseSent"], truncation=True, max_length=max_length, padding="max_length")
        labels = tokenizer(examples["reason"], truncation=True, max_length=max_length, padding="max_length")["input_ids"]
        inputs["labels"] = labels
        return inputs

In [12]:
train_dataset = train_dataset.map(lambda x: preprocess_data(x, tokenizer, max_length), batched=True)
dev_dataset = dev_dataset.map(lambda x: preprocess_data(x, tokenizer, max_length), batched=True)
test_dataset = test_dataset.map(lambda x: preprocess_data(x, tokenizer, max_length, True), batched=True)
print("Train formated Dataset example:\n")
print(train_dataset[0])
print("\nTest formated Dataset example:\n")
print(test_dataset[0])

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Train formated Dataset example:

{'id': 770, 'FalseSent': 'The plane is flying between the planets.', 'reason': 'The plane cannot go into space.', '__index_level_0__': 2307, 'input_ids': [0, 133, 3286, 16, 4731, 227, 5, 18979, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [0, 133, 3286, 1395, 213, 88, 980, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Test formated Dataset example:

{'id': 1280, 'FalseSent': 'Beer that is drunk by humans is white', 'reason1': 'Beer is made of barley and it is a yellow drink', 'reason2': 'A beer that is drunk by humans is not white.', 'reason3': 'Beer is brown', '__index_level_0__': 76, 'input_ids': [0, 45562, 14, 16, 10789, 30, 5868, 16, 1104, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


## Fine-tuning - [5 Marks]

In general, when using a `Trainer` to make predictions, it returns the logits for each class in the task. However, the `Seq2SeqTrainingArguments` class provides an option that allows the `Trainer` to generate sequences of tokens in the prediction. The `create_training_arguments` function must create the `Seq2SeqTrainingArguments` with that option and the hyperparamters passed as arguments. During the training, the model must be evaluated on the development set after every epoch. `TrainingArguments` should include this strategy.

> **Important!** By default, `Trainer` saves a checkpoint of the model every 500 training steps. For this assignment, avoid this behavior by setting `save_strategy="no"` when creating the `TrainingArguments`.

In [13]:
def create_training_arguments(epochs, train_batch_size, learning_rate, output_dir):   # [1 Mark]
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",  
        per_device_train_batch_size=train_batch_size,
        save_strategy="no",  
        predict_with_generate=True,  
        num_train_epochs=epochs,
        learning_rate=learning_rate,
    )
    return training_args

In [14]:
train_args = create_training_arguments(epochs, train_batch_size, learning_rate, output_dir)

In [15]:
from transformers import Seq2SeqTrainingArguments, TrainingArguments
from transformers import Trainer

Next, you can create a `Trainer` object initializing the appropriate data collator.

In [16]:
def create_trainer(model, train_args, train_dataset, dev_dataset, tokenizer):   # [1 Mark]
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    trainer = Trainer(
        model=model,
        args=train_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
    )
    return trainer

In [17]:
trainer = create_trainer(model, train_args, train_dataset, dev_dataset, tokenizer)

In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,8.284785
2,No log,7.718569
3,No log,7.562421


TrainOutput(global_step=36, training_loss=8.91283925374349, metrics={'train_runtime': 292.3144, 'train_samples_per_second': 0.924, 'train_steps_per_second': 0.123, 'total_flos': 4019258880000.0, 'train_loss': 8.91283925374349, 'epoch': 3.0})

If you have set the `Seq2SeqTrainingArguments` properly, you could now use the `Trainer` to predict sequences of tokens. Take into account that `Trainer` will return the indexes of the tokens, so the sequence must be decoded to obtain the text strings. The `tokenizer` provides functionality to do this. The result of this process can be stored in the `prediction` column of the test `DataFrame`:

|     |   id | FalseSent                                      | reason1                                                  | reason2                                                | reason3                                                            | prediction                                     |
|----:|-----:|:-----------------------------------------------|:---------------------------------------------------------|:-------------------------------------------------------|:-------------------------------------------------------------------|:-----------------------------------------------|
|  76 | 1280 | Beer that is drunk by humans is white          | Beer is made of barley and it is a yellow drink          | A beer that is drunk by humans is not white.           | Beer is brown                                                      | Beer that is drunk by humans is white                             |
| 101 |  860 | eating trash food every day makes you stronger | eating trash food every day makes your body fat and weak | eating trash food every day is bad for your health     | Trash food could be contaminated                                   | eating trash food every day makes you stronger |
| 136 |  777 | he put some cooking oil in his wine            | cooking oil will destroy the taste of the wine           | Cooking oil does not go in wine                        | Cooking oil does not taste nice and therefore would ruin the wine. | he put some cooking oil in his wine            |
| 174 |  570 | Lobsters live in the mountains                 | Lobsters needs water to live                             | Lobsters live in the sea.                              | Lobsters live in the sea, not the mountains                        | Lobsters live in mountains                 |
| 210 | 1929 | the clock shows animals                        | the clock is used to show the time to people             | Clocks are required to tell the time, not show animals | a clock shows the time not animals                                 | the clock shows animals                        |
| 235 | 1619 | she put the giraffe in the freezer             | A giraffe is much bigger than the freezer                | There is no way a giraffe is fitting in the freezer.   | A giraffe is too big to be put in a freezer.                       | she put the giraffe in the freezer             |


In [19]:
def make_predictions(trainer, test_dataset, tokenizer):   # [1 Mark]

    predictions = trainer.predict(test_dataset)
    logits = predictions.predictions[0]
    predicted_indexes = logits.argmax(-1)
    decoded_predictions = tokenizer.batch_decode(predicted_indexes, skip_special_tokens=True)
    test_data["prediction"] = decoded_predictions

    return decoded_predictions

In [20]:
predictions = make_predictions(trainer, test_dataset, tokenizer)
test_data["prediction"] = predictions
test_data

,id,FalseSent,reason1,reason2,reason3,prediction
76,1280,Beer that is drunk by humans is white,Beer is made of barley and it is a yellow drink,A beer that is drunk by humans is not white.,Beer is brown,Beer that is drunk by humans is whitec--
101,860,eating trash food every day makes you stronger,eating trash food every day makes your body fat and weak,eating trash food every day is bad for your health,Trash food could be contaminated,eating trash food every day makes you stronger--ACCACACACACACACACAC
136,777,he put some cooking oil in his wine,cooking oil will destroy the taste of the wine,Cooking oil does not go in wine,Cooking oil does not taste nice and therefore would ruin the wine.,he put some cooking oil in his wineCCCC
174,570,Lobsters live in the mountains,Lobsters needs water to live,Lobsters live in the sea.,"Lobsters live in the sea, not the mountains",Lobsters live in the mountains--------------
210,1929,the clock shows animals,the clock is used to show the time to people,"Clocks are required to tell the time, not show animals",a clock shows the time not animals,the clock shows animalsCCCCCACEACEACEAPEAPE
235,1619,she put the giraffe in the freezer,A giraffe is much bigger than the freezer,There is no way a giraffe is fitting in the freezer.,A giraffe is too big to be put in a freezer.,she put the giraffe in the freezerCCCCcccC
280,979,he installed the carpet on the lake,The carpet will absorb water and sink,Carpets need a subfloor.,A lake would not be able to grip onto carpet.,he installed the carpet on the lake andCCCCCCCCCcccC
319,75,My son had us write an essay on The National Monument.,My son isn't smart enough to assign an essay.,My son is studying in the seconds standard only,Children don't ask parents to write essays.,My son had us write an essay on The National Monument.cccccc
371,1810,He drove up the stairs to the bedroom,A car is too large to fit upstairs,Stairs are too small and weak for a car to drive up,People don't drive up indoor stairs.,He drove up the stairs to the bedroom andcccc
411,774,he put a piece of plastic on the bread,the plastic usually is toxic,You can't eat plastic.,People do not eat plastic because it's not a food,he put a piece of plastic on the breadCCCCCCC-C


The **Subtasks B** of **ComVE** is evaluated using the *bleu* metric. In this assignment, you will also evaluate using *rouge*. With `shrink_dataset` and `base_model` set to `True`, the expected scores are *0.216* and *0.446* for *bleu* and *rouge* respectively. With a full training run, i.e. with `shrink_dataset` and `base_model` set to `False`, the scores should be around *0.228* and *0.461*.

In [21]:
pip install sacrebleu

In [22]:
pip install rouge

Note: you may need to restart the kernel to use updated packages.


In [23]:
import sacrebleu
from rouge import Rouge

In [24]:
def evaluate_bleu(reference, candidate):   # [2 Marks]
    references = [' '.join(ref) for ref in reference]
    candidate = ' '.join(candidate.split())
    bleu = sacrebleu.corpus_bleu(candidate, references)
    return bleu.score

def evaluate_rouge(reference, candidate):
    rouge = Rouge()
    scores = rouge.get_scores(candidate, reference)
    return scores[0]['rouge-1']['f']

def evaluate_prediction(test_data, metric):
    references_data = test_data[['reason1', 'reason2', 'reason3']]
    references = references_data.values.tolist()
    predictions = test_data['prediction'].tolist()

    scores = []

    for reference, prediction in zip(references, predictions):
        if metric == 'bleu':
            score = evaluate_bleu(reference, prediction)
        elif metric == 'rouge':
            
            reference_string = ' '.join(reference)
            score = evaluate_rouge(reference_string, prediction)
        else:
            raise ValueError("Invalid metric. Choose 'bleu' or 'rouge'.")

        scores.append(score)

    return scores

In [25]:
 evaluate_prediction(test_data, "bleu")

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [26]:
evaluate_prediction(test_data, "rouge")

[0.46153845760355033,
 0.44444444027434843,
 0.23076922650887582,
 0.49999999570312514,
 0.2857142826303855,
 0.31999999635200005,
 0.14814814430727036,
 0.29411764249134953,
 0.20689654772889424,
 0.24999999531250006,
 0.11111110617283973,
 0.47999999564800006,
 0.23076922766272187,
 0.23076922721893492,
 0.39999999564800004,
 0.21621621253469692,
 0.3529411728719723,
 0.08695651788279792,
 0.239999996352,
 0.0,
 0.0,
 0.42857142482142857,
 0.062499997363281365,
 0.1904761854875285,
 0.3478260824196598,
 0.19999999680000002,
 0.23529411404844294,
 0.11428571144489805,
 0.3999999962500001,
 0.1538461499112427]

The scores for the partial training and the full training are so similar that it would appear that the full training does not provide any benefit in this task. However, it should be noted that the test sets in the two cases are different. More importantly, these results are indicative of the limitations of metrics such as *bleu* and *rouge* for evaluating text generation. Take, for example, the following case from the test set:


| FalseSent                 | reason1                                        | reason2                          | reason3                         |
|:--------------------------|:-----------------------------------------------|:---------------------------------|:--------------------------------|
| Beer that is drunk by humans is white | Beer is made of barley and it is a yellow drink | A beer that is drunk by humans is not white. | Beer is brown |

The predictions obtained by the partial and full trainings and their corresponding scores are the following:

| full training    | prediction                 | bleu     | rouge    |
|:-----------------|:---------------------------|---------:|---------:|
| no               | Beer that is drunk by humans is white  | 0.731    | 0.889    |
| yes              | White beer is not suitable for human consumption. | 0.000    | 0.364    |

The text generated by the full training is a better explanation than the reason generated by the partial training, which is a mere repetition of the nonsensical statement. However, the latter obtains much better scores than the former. Metrics such as *bleu* and *rouge* do not always replace accurately the human judgement.

In [27]:
blue_result =  evaluate_prediction(test_data, "bleu")
rouge_result = evaluate_prediction(test_data, "rouge")

test_data['blue'] = blue_result
test_data['rouge'] = rouge_result
test_data

,id,FalseSent,reason1,reason2,reason3,prediction,blue,rouge
76,1280,Beer that is drunk by humans is white,Beer is made of barley and it is a yellow drink,A beer that is drunk by humans is not white.,Beer is brown,Beer that is drunk by humans is whitec--,0.0,0.461538
101,860,eating trash food every day makes you stronger,eating trash food every day makes your body fat and weak,eating trash food every day is bad for your health,Trash food could be contaminated,eating trash food every day makes you stronger--ACCACACACACACACACAC,0.0,0.444444
136,777,he put some cooking oil in his wine,cooking oil will destroy the taste of the wine,Cooking oil does not go in wine,Cooking oil does not taste nice and therefore would ruin the wine.,he put some cooking oil in his wineCCCC,0.0,0.230769
174,570,Lobsters live in the mountains,Lobsters needs water to live,Lobsters live in the sea.,"Lobsters live in the sea, not the mountains",Lobsters live in the mountains--------------,0.0,0.500000
210,1929,the clock shows animals,the clock is used to show the time to people,"Clocks are required to tell the time, not show animals",a clock shows the time not animals,the clock shows animalsCCCCCACEACEACEAPEAPE,0.0,0.285714
235,1619,she put the giraffe in the freezer,A giraffe is much bigger than the freezer,There is no way a giraffe is fitting in the freezer.,A giraffe is too big to be put in a freezer.,she put the giraffe in the freezerCCCCcccC,0.0,0.320000
280,979,he installed the carpet on the lake,The carpet will absorb water and sink,Carpets need a subfloor.,A lake would not be able to grip onto carpet.,he installed the carpet on the lake andCCCCCCCCCcccC,0.0,0.148148
319,75,My son had us write an essay on The National Monument.,My son isn't smart enough to assign an essay.,My son is studying in the seconds standard only,Children don't ask parents to write essays.,My son had us write an essay on The National Monument.cccccc,0.0,0.294118
371,1810,He drove up the stairs to the bedroom,A car is too large to fit upstairs,Stairs are too small and weak for a car to drive up,People don't drive up indoor stairs.,He drove up the stairs to the bedroom andcccc,0.0,0.206897
411,774,he put a piece of plastic on the bread,the plastic usually is toxic,You can't eat plastic.,People do not eat plastic because it's not a food,he put a piece of plastic on the breadCCCCCCC-C,0.0,0.250000
